Reference: https://www.tensorflow.org/tutorials/customization/custom_layers?hl=en

Tensorflow v2.11.0 Module: tf.keras.layers: https://www.tensorflow.org/api_docs/python/tf/keras/layers
(Find all layers here)

In [29]:
import tensorflow as tf

In [30]:
print(tf.config.list_physical_devices('GPU'))

[]


In [31]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [32]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [33]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.28603992,  0.47989613,  0.44830233, -0.40685266, -0.23194328,
          0.47046334,  0.30157816,  0.3492518 ,  0.2225402 , -0.00219339],
        [ 0.24086517,  0.53062004, -0.08906257,  0.620475  ,  0.4428888 ,
          0.37213045, -0.5293078 ,  0.00153172, -0.2248331 ,  0.5714373 ],
        [ 0.19474119, -0.39436007,  0.5340995 , -0.20120656,  0.61623293,
         -0.02697331, -0.13377908, -0.13484123, -0.567585  , -0.40109837],
        [ 0.1585058 , -0.44142106, -0.36443657, -0.05609822, -0.09447241,
         -0.14501527, -0.18978655,  0.4140101 , -0.3648529 , -0.23976952],
        [-0.1069389 , -0.23282585,  0.24539077,  0.24390614,  0.57694954,
          0.03698075,  0.20816094, -0.07245374, -0.62996376,  0.14679444]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [34]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.28603992,  0.47989613,  0.44830233, -0.40685266, -0.23194328,
          0.47046334,  0.30157816,  0.3492518 ,  0.2225402 , -0.00219339],
        [ 0.24086517,  0.53062004, -0.08906257,  0.620475  ,  0.4428888 ,
          0.37213045, -0.5293078 ,  0.00153172, -0.2248331 ,  0.5714373 ],
        [ 0.19474119, -0.39436007,  0.5340995 , -0.20120656,  0.61623293,
         -0.02697331, -0.13377908, -0.13484123, -0.567585  , -0.40109837],
        [ 0.1585058 , -0.44142106, -0.36443657, -0.05609822, -0.09447241,
         -0.14501527, -0.18978655,  0.4140101 , -0.3648529 , -0.23976952],
        [-0.1069389 , -0.23282585,  0.24539077,  0.24390614,  0.57694954,
          0.03698075,  0.20816094, -0.07245374, -0.62996376,  0.14679444]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

### Implementing Custom Layers

The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

* __init__ , where you can do all input-independent initialization
* build, where you know the shapes of the input tensors and can do the rest of the initialization
* call, where you do the forward computation

Note that you don't have to wait until build is called to create your variables, you can also create them in __init__. However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.

In [35]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)   # Matrix multiplication for inputs

layer = MyDenseLayer(10)

In [36]:
_ = layer(tf.zeros([10, 5]))   # Calling the layer '.build's it

In [37]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer_1/kernel:0']


In [38]:
print([var.name for var in layer.trainable_weights])

['my_dense_layer_1/kernel:0']


In [39]:
print([var.name for var in layer.non_trainable_variables])

[]


#### Composing Layers

In [40]:
# ResNet block
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, kernel_size)  # Conv. layer
    self.bn2a = tf.keras.layers.BatchNormalization()        # Batch normalization layer

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same') # Conv. layer
    self.bn2b = tf.keras.layers.BatchNormalization()                            # Batch normalization layer

    self.conv2c = tf.keras.layers.Conv2D(filters3, kernel_size)  # Conv. layer
    self.bn2c = tf.keras.layers.BatchNormalization()        # Batch normalization layer

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [41]:
_ = block(tf.zeros([1, 2, 3, 3]), True)

In [42]:
block.trainable_variables

[<tf.Variable 'conv2d_6/kernel:0' shape=(1, 1, 3, 1) dtype=float32, numpy=
 array([[[[-1.1158684],
          [ 0.5235896],
          [ 1.2136856]]]], dtype=float32)>,
 <tf.Variable 'conv2d_6/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/gamma:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/beta:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'conv2d_7/kernel:0' shape=(1, 1, 1, 2) dtype=float32, numpy=array([[[[-0.76285964,  1.0807406 ]]]], dtype=float32)>,
 <tf.Variable 'conv2d_7/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_7/gamma:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_7/beta:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'conv2d_8/kernel:0' shape=(1, 1, 2, 3) dtype=float32, numpy=
 ar

In [43]:
len(block.variables)

18

In [44]:
block.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_6 (Batc  multiple                 4         
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_7 (Batc  multiple                 8         
 hNormalization)                                                 
                                                                 
 conv2d_8 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_8 (Batc  multiple                 12        


Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using tf.keras.Sequential

In [45]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [46]:
my_seq.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, None, None, 1)     4         
                                                                 
 batch_normalization_9 (Batc  (None, None, None, 1)    4         
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, None, None, 2)     4         
                                                                 
 batch_normalization_10 (Bat  (None, None, None, 2)    8         
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          (None, None, None, 3)     9         
                                                                 
 batch_normalization_11 (Bat  (None, None, None, 3)   

Reference: https://www.tensorflow.org/tutorials/customization/custom_layers?hl=en